In [ ]:
%matplotlib inline

# Load surface mask and distance matrix

import glob
import h5py
import numpy as np
from brainspace.gradient import GradientMaps
from brainspace.gradient import alignment
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.decomposition import PCA
import helpers

HCPFilenamesList = glob.glob('../IndSubHCP/*.mat')
HCPFilenamesList.sort()
SurfaceMask=np.loadtxt('surfBilateralMask.txt')

numFiles=len(HCPFilenamesList)
numV_l=np.count_nonzero(SurfaceMask[:5000])
numV_l=np.count_nonzero(SurfaceMask[5000:])

num_bins=20
dist_mat_file = "SurfL10kConte.txt"

dist_file=np.loadtxt(dist_mat_file)

dist_file_masked=dist_file[np.ix_(SurfaceMask[:5000]==1,SurfaceMask[:5000]==1)]

dist_file_masked=dist_file[np.ix_(SurfaceMask[:5000]==1,SurfaceMask[:5000]==1)]
reduceIndices=np.ix_(np.arange(0,dist_file_masked.shape[0],10),np.arange(0,dist_file_masked.shape[0],10))
dist_file_masked_reduced=dist_file_masked[reduceIndices]



In [ ]:
# Decimate the data to make the smoothing/resampling step computationally tractable on laptop
reduceIndices=np.ix_(np.arange(0,dist_file_masked.shape[0],10),np.arange(0,dist_file_masked.shape[0],10))
dist_file_masked_reduced=dist_file_masked[reduceIndices]


In [ ]:
%matplotlib inline

# Load individual FMRI data on surface and calculate group variogram

import glob
import h5py
import numpy as np
from brainspace.gradient import GradientMaps
from brainspace.gradient import alignment
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.decomposition import PCA
import helpers

HCPFilenamesList = glob.glob('../IndSubHCP/*.mat')
HCPFilenamesList.sort()
SurfaceMask=np.loadtxt('surfBilateralMask.txt')

numFiles=len(HCPFilenamesList)
numV_l=np.count_nonzero(SurfaceMask[:5000])
numV_l=np.count_nonzero(SurfaceMask[5000:])

num_bins=20
dist_mat_file = "SurfL10kConte.txt"
dist_mat_file_r = "SurfR10kConte.txt"
dist_file=np.loadtxt(dist_mat_file)
dist_file_r=np.loadtxt(dist_mat_file_r)
dist_file_masked=dist_file[np.ix_(SurfaceMask[:5000]==1,SurfaceMask[:5000]==1)]
dist_file_masked_r=dist_file_r[np.ix_(SurfaceMask[5000:]==1,SurfaceMask[5000:]==1)]


FC_l=np.zeros([dist_file_masked.shape[0],dist_file_masked.shape[0]])
FC_r=np.zeros([dist_file_masked_r.shape[0],dist_file_masked_r.shape[0]])
ind_fc_ev_l=np.zeros([numFiles,num_bins])
ind_fc_h_l=np.zeros([numFiles,num_bins])
ind_fc_ev_r=np.zeros([numFiles,num_bins])
ind_fc_h_r=np.zeros([numFiles,num_bins])
ind_fc_ev_l_2=np.zeros([numFiles,num_bins])
ind_fc_h_l_2=np.zeros([numFiles,num_bins])
ind_fc_ev_r_2=np.zeros([numFiles,num_bins])
ind_fc_h_r_2=np.zeros([numFiles,num_bins])

for i,dataName in enumerate(HCPFilenamesList):



    print(dataName)
    f = h5py.File(dataName)
    arrays = {}
    for k, v in f.items():
        arrays[k] = np.array(v)

    IndData1=arrays['rfMRI_REST1_LR']
    
    tempData=IndData1[:5000,:]
    tempDataReduced=tempData[SurfaceMask[:5000]==1,:]
    tempFC=np.corrcoef(tempDataReduced)
    ind_fc_h_l[i,:],ind_fc_ev_l[i,:]=helpers.emp_variogram_conn(tempFC,dist_file_masked,num_bins,[0,150])
    tempFC=np.arctanh(tempFC)
    np.fill_diagonal(tempFC,3)
    FC_l=FC_l+tempFC
    
    tempData=IndData1[5000:,:]
    tempDataReduced=tempData[SurfaceMask[5000:]==1,:]
    tempFC=np.corrcoef(tempDataReduced)
    ind_fc_h_r[i,:],ind_fc_ev_r[i,:]=helpers.emp_variogram_conn(tempFC,dist_file_masked_r,num_bins,[0,150])
    tempFC=np.arctanh(tempFC)
    np.fill_diagonal(tempFC,3)
    FC_r=FC_r+tempFC
    
    IndData1=arrays['rfMRI_REST1_RL']
    tempData=IndData1[:5000,:]
    tempDataReduced=tempData[SurfaceMask[:5000]==1,:]
    tempFC=np.corrcoef(tempDataReduced)
    ind_fc_h_l_2[i,:],ind_fc_ev_l_2[i,:]=helpers.emp_variogram_conn(tempFC,dist_file_masked,num_bins,[0,150])

    
    tempData=IndData1[5000:,:]
    tempDataReduced=tempData[SurfaceMask[5000:]==1,:]
    tempFC=np.corrcoef(tempDataReduced)
    ind_fc_h_r_2[i,:],ind_fc_ev_r_2[i,:]=helpers.emp_variogram_conn(tempFC,dist_file_masked_r,num_bins,[0,150])
    
    


FC_l=FC_l/(numFiles+1)
FC_l=np.tanh(FC_l)   

FC_r=FC_r/(numFiles+1)
FC_r=np.tanh(FC_r)  



In [ ]:
# Resample function
def replace_corr_values(smoothed_corr, original_corr):
    # Rank the values in the smoothed and original correlation matrices
    smoothed_ranks = np.argsort(np.argsort(smoothed_corr, axis=None)).reshape(smoothed_corr.shape)
    original_ranks = np.argsort(np.argsort(original_corr, axis=None)).reshape(original_corr.shape)
    # Replace the values in the smoothed correlation matrix with values from the original correlation matrix based on rank
    for row in range(smoothed_corr.shape[0]):
        for col in range(row + 1, smoothed_corr.shape[1]):
            if smoothed_corr[row, col] < 1.0:
                smoothed_rank = smoothed_ranks[row, col]
                original_ranked_values = original_corr[original_ranks == smoothed_rank]
                original_ranked_values = original_ranked_values[original_ranked_values < 1.0]
                if original_ranked_values.size > 0:
                    original_value = original_ranked_values[-1]
                    smoothed_corr[row, col] = original_value
                    smoothed_corr[col, row] = original_value
    # Set the diagonal elements to 1
    np.fill_diagonal(smoothed_corr, 1)
    return smoothed_corr

# Gaussian smoothing function
def smooth_corr(corr, dist, k=5, b=1.0,Resample=False):
    # Sort distances and get indices of k-nearest neighbors
    knn_indices = np.argsort(dist, axis=1)[:, :k]
    # Compute weights for each edge using exponential kernel
    weights = np.exp(-dist/dist.max() / b)
    # Set weights to 0 for edges not in k-nearest neighbors
    mask = np.zeros_like(weights)
    mask[np.arange(len(mask))[:, None], knn_indices] = 1
    weights *= mask
    # Normalize weights for each node to sum to 1
    row_sums = weights.sum(axis=1)
    weights /= row_sums[:, np.newaxis]
    # Compute smoothed correlation matrix
    smoothed_corr = weights.dot(corr).dot(weights.T)

    smoothed_corr = (smoothed_corr + smoothed_corr.T) / 2
    np.fill_diagonal(smoothed_corr, smoothed_corr.max())
    
    if Resample:
        smoothed_corr = replace_corr_values(smoothed_corr, corr)
    return smoothed_corr


In [ ]:
# Find best smoothing parameters for null model to approximately fit empirical variogram

from scipy import special
from scipy.optimize import curve_fit
import math
h_l,wb_average_ev_l=helpers.emp_variogram_conn(EmpFC,dist_file_masked_reduced,num_bins,[0,150])


EmpFC=FC_l[reduceIndices]
mat=EmpFC
nr = mat.shape[0] # number of regions
mat_rand = np.zeros(mat.shape) # initialise number of regions
mat_rand[np.triu_indices(nr,1)] = np.random.permutation(mat[np.triu_indices(nr,1)]) # permute upper triangular
mat_rand = mat_rand+mat_rand.T # fill lower triangular
mat_rand[np.diag_indices(nr)] = 1 # set diagonal to 1
mantel_ind = np.random.permutation(nr) # indices to permute regions
FC_mantel = mat[mantel_ind,:][:,mantel_ind] # apply to both rows and columns
FC_mantel = (FC_mantel+FC_mantel.T)/2 # fill lower triangular


cutoff=np.linspace(30,90,20,dtype=int)
smooth=np.linspace(0.02,0.05,20)
optSmooth=np.zeros([len(cutoff),len(smooth)])
for i,cut in enumerate(cutoff):
    for j, sm in enumerate(smooth):
        mantel_ind = np.random.permutation(nr) # indices to permute regions
        FC_mantel = mat[mantel_ind,:][:,mantel_ind] # apply to both rows and columns
        FC_mantel = (FC_mantel+FC_mantel.T)/2 # fill lower triangular


        smoothed_corr_mat=smooth_corr(FC_mantel,dist_file_masked_reduced,cut,sm,True)
        h,Emp_vg=helpers.emp_variogram_conn(EmpFC,dist_file_masked_reduced,num_bins,[0,150])
        h,vg=helpers.emp_variogram_conn(smoothed_corr_mat,dist_file_masked_reduced,num_bins,[0,150])
        optSmooth[i,j]= np.power(np.power((vg-wb_average_ev_l),2).sum(),0.5)
        


In [ ]:
# Take optimal parameters from above and generate 1000 Mantel shuffled, smoothed and resampled datasets

reduceIndices=np.ix_(np.arange(0,dist_file_masked.shape[0],10),np.arange(0,dist_file_masked.shape[0],10))
dist_file_masked_reduced=dist_file_masked[reduceIndices]
EmpFC=FC_l[reduceIndices]

mat=EmpFC.copy()


nr = mat.shape[0] # number of regions
h_l,wb_average_ev_l=helpers.emp_variogram_conn(EmpFC,dist_file_masked_reduced,num_bins,[0,150])

perms=1000
#AllVGRandSmoothed=np.zeros([wb_average_ev_l.shape[0],perms])
AllMatRandUnsmoothed1=np.zeros([EmpFC.shape[0],EmpFC.shape[1],perms])
AllMatRandUnsmoothed2=np.zeros([EmpFC.shape[0],EmpFC.shape[1],perms])
#AllMatRandSmoothed=np.zeros([EmpFC.shape[0],EmpFC.shape[1],perms])

#AllSmoothRegress_betas=np.zeros([2,perms])

cut=45 # Parameters 
sm=0.03
for i in range(perms):
    if np.mod(i,20)==0:
        print(i)
    mat_rand = np.zeros(mat.shape) # initialise number of regions
    mat_rand[np.triu_indices(nr,1)] = np.random.permutation(mat[np.triu_indices(nr,1)]) # permute upper triangular
    mat_rand = mat_rand+mat_rand.T # fill lower triangular
    mat_rand[np.diag_indices(nr)] = 1 # set diagonal to 1
    mantel_ind = np.random.permutation(nr) # indices to permute regions
    FC_mantel = mat[mantel_ind,:][:,mantel_ind] # apply to both rows and columns
    FC_mantel = (FC_mantel+FC_mantel.T)/2 # fill lower triangular
    FC_mantel[np.diag_indices(nr)] = 1 # set diagonal to 1
    AllMatRandUnsmoothed1[:,:,i]=FC_mantel
    AllMatRandUnsmoothed2[:,:,i]=mat_rand
    


In [ ]:
# Plot different null model variograms and empirical variogram
h,Emp_vg=helpers.emp_variogram_conn(EmpFC,dist_file_masked_reduced,num_bins,[0,150])
h,vg=helpers.emp_variogram_conn(AllMatRandSmoothed[:,:,23],dist_file_masked_reduced,num_bins,[0,150])
h,vgMant=helpers.emp_variogram_conn(FC_mantel,dist_file_masked_reduced,num_bins,[0,150])
h,vgRand=helpers.emp_variogram_conn(mat_rand,dist_file_masked_reduced,num_bins,[0,150])


fig=plt.figure(figsize=(8, 8))
fig.patch.set_alpha(0)
smoothing=0.5
x=gaussian_filter1d(Emp_vg, sigma=smoothing)
plt.plot(h, x, 'o',c='b',label='Empirical')
x=gaussian_filter1d(vgRand, sigma=smoothing)
plt.plot(h, x, '--',c='r',label='Random permutation')
x=gaussian_filter1d(vgMant, sigma=smoothing)
plt.plot(h, x, '--',c='k',label='Mantel permutation')
x=gaussian_filter1d(vg, sigma=smoothing)
plt.plot(h, x, '--',c='g',label='Mantel permutation followed by smoothing')
plt.legend()
plt.savefig('./imagesAvG/NullVariograms.png')

In [ ]:
# Plot example connectivity matrices

fig, axs = plt.subplots(1,4,figsize=(16,4))
fig.patch.set_alpha(0)

#plt.imshow(AllMatRandSmoothed[:,:,23],interpolation='nearest')
axs[0].imshow(EmpFC,interpolation='nearest',cmap=plt.cm.coolwarm,vmin=-0.5,vmax=0.5)
axs[1].imshow(AllMatRandUnsmoothed2[:,:,1],interpolation='nearest',cmap=plt.cm.coolwarm,vmin=-0.5,vmax=0.5)
axs[2].imshow(AllMatRandUnsmoothed1[:,:,1],interpolation='nearest',cmap=plt.cm.coolwarm,vmin=-0.5,vmax=0.5)
axs[3].imshow(AllMatRandSmoothed[:,:,1],interpolation='nearest',cmap=plt.cm.coolwarm,vmin=-0.5,vmax=0.5)

plt.savefig('./imagesAvG/NullFCs.png')

In [ ]:
# Fit theoretical variograms to different null models

perms=1000

US1ShuffModelPred_evVw_l=np.zeros([perms,evVw_l.shape[0],4,num_bins-3])
US1ShuffModelSSE_evVw_l=np.zeros([perms,evVw_l.shape[0],4])
US1ShuffModelCof_evVw_l=np.zeros([perms,evVw_l.shape[0],4,3])

US2ShuffModelPred_evVw_l=np.zeros([perms,evVw_l.shape[0],4,num_bins-3])
US2ShuffModelSSE_evVw_l=np.zeros([perms,evVw_l.shape[0],4])
US2ShuffModelCof_evVw_l=np.zeros([perms,evVw_l.shape[0],4,3])


for i in range(perms):
    hVw_l,evVw_l=helpers.emp_variogram_vwise_conn(AllMatRandUnsmoothed1[:,:,i],dist_file_masked_reduced,num_bins,[0,150]) #vary last number for maximum range 

    for j in range(evVw_l.shape[0]):
        y=evVw_l[j,clipStart:clipEnd]
        x=hVw_l[j,clipStart:clipEnd]
        bounds = (0.00001, [np.nanmax(x),np.nanmax(y),np.nanmax(y)/10])

        cof = None
        cov = None

        for ind,model in enumerate(models):

            cof, cov = curve_fit(model,x, y, method='trf',p0=bounds[1],bounds=bounds,maxfev=5000)
            v_pred=model(x,*cof)
            
            US1ShuffModelPred_evVw_l[i,j,ind,:]=v_pred
            US1ShuffModelSSE_evVw_l[i,j,ind]=np.sum((v_pred-y)**2)
            US1ShuffModelCof_evVw_l[i,j,ind,:]=cof



for i in range(perms):
    hVw_l,evVw_l=helpers.emp_variogram_vwise_conn(AllMatRandUnsmoothed2[:,:,i],dist_file_masked_reduced,num_bins,[0,150]) #vary last number for maximum range 

    for j in range(evVw_l.shape[0]):
        y=evVw_l[j,clipStart:clipEnd]
        x=hVw_l[j,clipStart:clipEnd]
        bounds = (0.00001, [np.nanmax(x),np.nanmax(y),np.nanmax(y)/10])

        cof = None
        cov = None

        for ind,model in enumerate(models):

            cof, cov = curve_fit(model,x, y, method='trf',p0=bounds[1],bounds=bounds,maxfev=5000)
            v_pred=model(x,*cof)
            US2ShuffModelPred_evVw_l[i,j,ind,:]=v_pred
            US2ShuffModelSSE_evVw_l[i,j,ind]=np.sum((v_pred-y)**2)
            US2ShuffModelCof_evVw_l[i,j,ind,:]=cof
    

In [ ]:
# Plot correlation of different null permutations with Principal gradient
print(np.corrcoef(ModelCof_evVw_l[:,1,0],gml.gradients_[:,0])[0,1])
print(np.corrcoef(ModelCof_evVw_l[:,1,1],gml.gradients_[:,0])[0,1])
CorrGradRange=np.zeros([perms,3])
CorrGradSill=np.zeros([perms,3])
for i in range(perms):
    CorrGradRange[i,2]=np.corrcoef(ShuffModelCof_evVw_l[i,:,1,0],gml.gradients_[:,0])[0,1]
    CorrGradSill[i,2]=np.corrcoef(ShuffModelCof_evVw_l[i,:,1,1],gml.gradients_[:,0])[0,1]

for i in range(perms):
    CorrGradRange[i,1]=np.corrcoef(US1ShuffModelCof_evVw_l[i,:,1,0],gml.gradients_[:,0])[0,1]
    CorrGradSill[i,1]=np.corrcoef(US1ShuffModelCof_evVw_l[i,:,1,1],gml.gradients_[:,0])[0,1]

for i in range(perms):
    CorrGradRange[i,0]=np.corrcoef(US2ShuffModelCof_evVw_l[i,:,1,0],gml.gradients_[:,0])[0,1]
    CorrGradSill[i,0]=np.corrcoef(US2ShuffModelCof_evVw_l[i,:,1,1],gml.gradients_[:,0])[0,1]

print('Corr Max')
print('Range')
print(CorrGradRange.max(axis=0))
print('Sill')
print(CorrGradSill.max(axis=0))

print('Corr Std')

print('Range')
print(CorrGradRange.std(axis=0))
print('Sill')
print(CorrGradSill.std(axis=0))


print('Real STD')
print('Range')
print(np.std(ModelCof_evVw_l[:,1,0],axis=0))
print('Sill')
print(np.std(ModelCof_evVw_l[:,1,1],axis=0))

print('Std')
print('Range')
print(ShuffModelCof_evVw_l[:,:,1,0].std(axis=0).mean())
print(US1ShuffModelCof_evVw_l[:,:,1,0].std(axis=0).mean())
print(US2ShuffModelCof_evVw_l[:,:,1,0].std(axis=0).mean())
print('Sill')
print(ShuffModelCof_evVw_l[:,:,1,1].std(axis=0).mean())
print(US1ShuffModelCof_evVw_l[:,:,1,1].std(axis=0).mean())
print(US2ShuffModelCof_evVw_l[:,:,1,1].std(axis=0).mean())


fig = plt.figure()
fig.patch.set_alpha(0)

ax = fig.add_axes([0.1,0.1,0.9,0.9])
bp = ax.violinplot(CorrGradRange)

for pc in bp['bodies']:
    pc.set_facecolor('white')
    pc.set_edgecolor('black')
    
for partname in ('cbars','cmins','cmaxes'):
    vp = bp[partname]
    vp.set_edgecolor('k')
    vp.set_linewidth(1)
ax.axhline(np.corrcoef(ModelCof_evVw_l[:,1,0],gml.gradients_[:,0])[0,1],ls="--", color="k")

ax.set_xticks([1, 2,3])
ax.set_xticklabels(["Random","Mantel","Smoothed Mantel"])
plt.savefig('./imagesAvG/NullCorrGradRange.png')
plt.show()


fig = plt.figure()
fig.patch.set_alpha(0)

ax = fig.add_axes([0.1,0.1,0.9,0.9])
bp = ax.violinplot(CorrGradSill)

for pc in bp['bodies']:
    pc.set_facecolor('white')
    pc.set_edgecolor('black')
    
for partname in ('cbars','cmins','cmaxes'):
    vp = bp[partname]
    vp.set_edgecolor('k')
    vp.set_linewidth(1)
ax.axhline(np.corrcoef(ModelCof_evVw_l[:,1,1],gml.gradients_[:,0])[0,1],ls="--", color="k")

ax.set_xticks([1, 2, 3])
ax.set_xticklabels(["Random","Mantel","Smoothed Mantel"])
plt.savefig('./imagesAvG/NullCorrGradSill.png')
plt.show()


fig = plt.figure()
fig.patch.set_alpha(0)
data_to_plot = [US2ShuffModelCof_evVw_l[:,:,1,0].std(axis=0), US1ShuffModelCof_evVw_l[:,:,1,0].std(axis=0),ShuffModelCof_evVw_l[:,:,1,0].std(axis=0) ]
ax = fig.add_axes([0.1,0.1,0.9,0.9])
bp = ax.violinplot(data_to_plot)

for pc in bp['bodies']:
    pc.set_facecolor('white')
    pc.set_edgecolor('black')
    
for partname in ('cbars','cmins','cmaxes'):
    vp = bp[partname]
    vp.set_edgecolor('k')
    vp.set_linewidth(1)
ax.axhline(np.std(ModelCof_evVw_l[:,1,0],axis=0),ls="--", color="k")

ax.set_xticks([1, 2,3])
ax.set_xticklabels(["Random","Mantel","Smoothed Mantel"])
plt.savefig('./imagesAvG/NullSTDRange.png')
plt.show()


fig = plt.figure()
fig.patch.set_alpha(0)
data_to_plot = [US2ShuffModelCof_evVw_l[:,:,1,1].std(axis=0), US1ShuffModelCof_evVw_l[:,:,1,1].std(axis=0),ShuffModelCof_evVw_l[:,:,1,1].std(axis=0) ]
ax = fig.add_axes([0.1,0.1,0.9,0.9])
bp = ax.violinplot(data_to_plot)

for pc in bp['bodies']:
    pc.set_facecolor('white')
    pc.set_edgecolor('black')
    
for partname in ('cbars','cmins','cmaxes'):
    vp = bp[partname]
    vp.set_edgecolor('k')
    vp.set_linewidth(1)
ax.axhline(np.std(ModelCof_evVw_l[:,1,1],axis=0),ls="--", color="k")

ax.set_xticks([1, 2,3])
ax.set_xticklabels(["Random","Mantel","Smoothed Mantel"])

plt.savefig('./imagesAvG/NullSTDSill.png')

plt.show()



In [ ]:
# Plot the relationship between sill/range and principal gradient for each null and empirical data
from brainspace.gradient import GradientMaps
from brainspace.gradient import alignment

numComp=3

#gmr = GradientMaps(n_components=numComp, approach='dm', kernel='normalized_angle')
gml = GradientMaps(n_components=numComp,approach='le')

gml.fit(EmpFC)
import matplotlib

matplotlib.rc_file_defaults()
#print(np.corrcoef(ModelCof_IndWBev_l[:,1,0],ModelCof_IndWBev_l_2[:,1,0]))

fig, axs = plt.subplots(1,4,figsize=(18,4))
fig.patch.set_alpha(0)
#print(stats.spearmanr(ModelCof_IndWBev_l[:,1,0],ModelCof_IndWBev_l_2[:,1,0]))

axs[0].scatter(ModelCof_evVw_l[:,1,0],ModelCof_evVw_l[:,1,1],c=gml.gradients_[:,0],cmap='plasma',s=5)
axs[0].grid(False)

it=10
axs[3].scatter(ShuffModelCof_evVw_l[it,:,1,0],ShuffModelCof_evVw_l[it,:,1,1],c=gml.gradients_[:,0],cmap='plasma',s=5)
axs[3].grid(False)

axs[2].scatter(US1ShuffModelCof_evVw_l[it,:,1,0],US1ShuffModelCof_evVw_l[it,:,1,1],c=gml.gradients_[:,0],cmap='plasma',s=5)
axs[2].grid(False)

axs[1].scatter(US2ShuffModelCof_evVw_l[it,:,1,0],US2ShuffModelCof_evVw_l[it,:,1,1],c=gml.gradients_[:,0],cmap='plasma',s=5)
axs[1].grid(False)

plt.savefig('./imagesAvG/NullExample.png')
#np.corrcoef(ModelCof_evVw_l[:,2,0],gml.gradients_[:,0])

#fig.colorbar(im,ax=axs[1])
#plt.savefig('./imagesAvG/RangeSill_Gradient_3.png')

#np.corrcoef(ModelCof_evVw_l[:,2,0],gml.gradients_[:,0])


#plt.scatter(ModelCof_evVw_l[:,1,0],ModelCof_evVw_l[:,1,1],c=gml.gradients_[:,0])